In [4]:
import pandas as pd

off_train = pd.read_csv('data/ccf_offline_stage1_train.csv',parse_dates=["Date"],header=0)
off_train.columns = ['user_id','merchant_id','coupon_id','discount_rate','distance','date_received','date']

off_test = pd.read_csv('data/ccf_offline_stage1_test_revised.csv',header=0)
off_test.columns = ['user_id','merchant_id','coupon_id','discount_rate','distance','date_received']

on_train = pd.read_csv('data/ccf_online_stage1_train.csv',parse_dates=["Date"],header=0)
on_train.columns = ['user_id','merchant_id','action','coupon_id','discount_rate','date_received','date']

# 【dataset3】20160701~20160731 (113640)   【feature3】20160315~20160630  （测试集）
dataset3 = off_test.dropna(how='any')
feature3 = off_train[((off_train.date >= '20160315') & (off_train.date <= '20160630'))
                   | ((off_train.date == 'null') & (off_train.date_received >= '20160315') 
                      &  (off_train.date_received <= '20160630'))
                   ].dropna(how='any')
# 【dataset2】20160515~20160615 (258446)   【feature2】20160201~20160514  （训练集2）
dataset2 = off_train[((off_train.date_received >= '20160515') & (off_train.date_received <= '20160615'))].dropna(how='any')
feature2 = off_train[((off_train.date >= '20160201') & (off_train.date <= '20160514'))
                   | ((off_train.date == 'null') & (off_train.date_received >= '20160201') 
                      &  (off_train.date_received <= '20160514'))
                   ].dropna(how='any')
# 【dataset1】20160414~20160514 (138303)   【feature1】20160101~20160413  （训练集1）
dataset1 = off_train[((off_train.date_received >= '20160414') & (off_train.date_received <= '20160514'))].dropna(how='any')
feature1 = off_train[((off_train.date >= '20160101') & (off_train.date <= '20160413'))
                   | ((off_train.date == 'null') & (off_train.date_received >= '20160101') 
                      &  (off_train.date_received <= '20160413'))
                   ].dropna(how='any')

In [5]:
from datetime import date
import numpy as np

In [6]:
def merchantRelatedFeature(feature3 ,file_name):
    merchant3 = feature3[['merchant_id', 'coupon_id', 'distance', 'date_received', 'date']]
    merchant3.ix[:5]
    
    #total_sales # 显示每个商户的销售数量
    t = merchant3[['merchant_id']]
    t.drop_duplicates(inplace=True)

    t1 = merchant3[merchant3.date != 'null'][['merchant_id']]
    t1['total_sales'] = 1
    t1 = t1.groupby('merchant_id').agg('sum').reset_index()
    t1.ix[:5]
    
    #sales_use_coupon # 显示使用了优惠券消费的商品
    t2 = merchant3[(merchant3.date != 'null') & (merchant3.coupon_id != 'null')][['merchant_id']]
    t2['sales_use_coupon'] = 1
    t2 = t2.groupby('merchant_id').agg('sum').reset_index()
    t2.ix[:5]
    
    # total_coupon # 商品的优惠券的总数量
    t3 = merchant3[merchant3.coupon_id != 'null'][['merchant_id']]
    t3['total_coupon'] = 1
    t3 = t3.groupby('merchant_id').agg('sum').reset_index()
    t3.ix[:5]

    t4 = merchant3[(merchant3.date != 'null') & (merchant3.coupon_id != 'null')][['merchant_id', 'distance']]
    t4.replace('null', -1, inplace=True)
    t4.distance = t4.distance.astype('int')
    t4.replace(-1, np.nan, inplace=True)
    t4.ix[:5]
    # merchant_min_distance # 所有使用优惠券消费的用户与商户的距离最小值
    t5 = t4.groupby('merchant_id').agg('min').reset_index()
    t5.rename(columns={'distance': 'merchant_min_distance'}, inplace=True)
    t5.ix[:5]
    # merchant_max_distance # 所有使用优惠券消费的用户与商户的距离最大值
    t6 = t4.groupby('merchant_id').agg('max').reset_index()
    t6.rename(columns={'distance': 'merchant_max_distance'}, inplace=True)
    t6.ix[:5]
    # merchant_mean_distance # 所有使用优惠券消费的用户与商户的距离平均值
    t7 = t4.groupby('merchant_id').agg('mean').reset_index()
    t7.rename(columns={'distance': 'merchant_mean_distance'}, inplace=True)
    t7.ix[:5]
    #merchant_median_distance # 所有使用优惠券消费的用户与商户的距离中位值
    t8 = t4.groupby('merchant_id').agg('median').reset_index()
    t8.rename(columns={'distance': 'merchant_median_distance'}, inplace=True)
    t8.ix[:5]
    
    # 合并以上特征数据
    merchant3_feature = pd.merge(t, t1, on='merchant_id', how='left')
    merchant3_feature = pd.merge(merchant3_feature, t2, on='merchant_id', how='left')
    merchant3_feature = pd.merge(merchant3_feature, t3, on='merchant_id', how='left')
    merchant3_feature = pd.merge(merchant3_feature, t5, on='merchant_id', how='left')
    merchant3_feature = pd.merge(merchant3_feature, t6, on='merchant_id', how='left')
    merchant3_feature = pd.merge(merchant3_feature, t7, on='merchant_id', how='left')
    merchant3_feature = pd.merge(merchant3_feature, t8, on='merchant_id', how='left')
    
    merchant3_feature = merchant3_feature.replace(np.nan, 0)  # fillna with 0
    merchant3_feature.ix[:5]
    
    # coupon_rate = sales_use_coupon/total_sales # 卖出商品中使用优惠券的占比
    merchant3_feature['coupon_rate'] = merchant3_feature.sales_use_coupon/merchant3_feature.total_sales
    merchant3_feature.ix[:5]
    # merchant_coupon_transfer_rate = sales_use_coupon/total_coupon # 优惠券的使用率
    merchant3_feature['merchant_coupon_transfer_rate'] = merchant3_feature.sales_use_coupon/merchant3_feature.total_coupon
    merchant3_feature.ix[:5]
    merchant3_feature.to_csv(file_name, index=None)
    return merchant3_feature

In [7]:
merchant3_feature = merchantRelatedFeature(feature3 ,'data/merchant3_feature.csv')
merchant3_feature.ix[:5]

C:\Users\xhb_1\Anaconda3\lib\site-packages\pandas\util\decorators.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return func(*args, **kwargs)


,merchant_id,total_sales,sales_use_coupon,total_coupon,merchant_min_distance,merchant_max_distance,merchant_mean_distance,merchant_median_distance,coupon_rate,merchant_coupon_transfer_rate
0,4663,393.0,39.0,11445.0,0.0,10.0,1.911765,1.0,0.099237,0.003408
1,2632,8.0,3.0,23.0,0.0,1.0,0.666667,1.0,0.375000,0.130435
2,3381,9579.0,416.0,46729.0,0.0,10.0,1.713580,1.0,0.043428,0.008902
3,450,7757.0,1239.0,35933.0,0.0,10.0,0.894068,0.0,0.159727,0.034481
4,6459,20.0,0.0,16.0,0.0,0.0,0.000000,0.0,0.000000,0.000000
5,6901,8939.0,994.0,13886.0,0.0,10.0,0.557895,0.0,0.111198,0.071583


In [8]:
merchant2_feature = merchantRelatedFeature(feature2 ,'data/merchant2_feature.csv')
merchant2_feature.ix[:5]

C:\Users\xhb_1\Anaconda3\lib\site-packages\pandas\util\decorators.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return func(*args, **kwargs)


,merchant_id,total_sales,sales_use_coupon,total_coupon,merchant_min_distance,merchant_max_distance,merchant_mean_distance,merchant_median_distance,coupon_rate,merchant_coupon_transfer_rate
0,2632,14.0,3.0,31.0,1.0,1.0,1.000000,1.0,0.214286,0.096774
1,3381,11537.0,1980.0,56499.0,0.0,10.0,1.697002,1.0,0.171622,0.035045
2,2099,5711.0,1497.0,12215.0,0.0,10.0,1.004814,0.0,0.262126,0.122554
3,1569,493.0,91.0,25173.0,0.0,10.0,2.388235,1.0,0.184584,0.003615
4,8390,1207.0,133.0,690.0,0.0,10.0,0.865079,0.0,0.110191,0.192754
5,7884,1541.0,184.0,2686.0,0.0,10.0,1.180124,0.0,0.119403,0.068503


In [9]:
merchant1_feature = merchantRelatedFeature(feature1 ,'data/merchant1_feature.csv')
merchant1_feature.ix[:5]

C:\Users\xhb_1\Anaconda3\lib\site-packages\pandas\util\decorators.py:91: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return func(*args, **kwargs)


,merchant_id,total_sales,sales_use_coupon,total_coupon,merchant_min_distance,merchant_max_distance,merchant_mean_distance,merchant_median_distance,coupon_rate,merchant_coupon_transfer_rate
0,2632,14.0,1.0,28.0,1.0,1.0,1.000000,1.0,0.071429,0.035714
1,3381,14962.0,2168.0,100369.0,0.0,10.0,1.650457,1.0,0.144900,0.021600
2,2099,5783.0,1705.0,16824.0,0.0,10.0,0.968072,0.0,0.294830,0.101343
3,4833,522.0,116.0,8321.0,0.0,10.0,3.037736,2.0,0.222222,0.013941
4,8390,1056.0,133.0,690.0,0.0,10.0,0.865079,0.0,0.125947,0.192754
5,1041,2078.0,308.0,8058.0,0.0,10.0,1.052265,0.0,0.148219,0.038223
